In [38]:
import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *

from utils import load_and_preprocess

from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib

In [39]:
%matplotlib tk
location_id = 329 # 330, 331
path = 'data/hystreet_fussgaengerfrequenzen_seit2021.csv'
df = load_and_preprocess(path, location_id)
df


,temperature,pedestrians_count,is_holiday,is_special_day,date,weather_condition_clear-day,weather_condition_clear-night,weather_condition_cloudy,weather_condition_fog,weather_condition_partly-cloudy-day,weather_condition_partly-cloudy-night,weather_condition_rain,weather_condition_snow,weather_condition_wind,time_cos,time_sin,day_cos,day_sin,dayofyear_cos,dayofyear_sin
0,13.51,151,0,0,2021-09-28 22:00:00,0,0,1,0,0,0,0,0,0,0.866025,-5.000000e-01,0.623490,0.781831,-0.047321,-0.998880
3,13.46,19,0,0,2021-09-28 23:00:00,0,0,1,0,0,0,0,0,0,0.965926,-2.588190e-01,0.623490,0.781831,-0.047321,-0.998880
6,13.26,10,0,0,2021-09-29 00:00:00,0,0,0,0,0,0,1,0,0,1.000000,0.000000e+00,-0.222521,0.974928,-0.030120,-0.999546
9,13.51,6,0,0,2021-09-29 01:00:00,0,0,0,0,0,0,1,0,0,0.965926,2.588190e-01,-0.222521,0.974928,-0.030120,-0.999546
12,13.44,22,0,0,2021-09-29 02:00:00,0,0,0,0,0,0,1,0,0,0.866025,5.000000e-01,-0.222521,0.974928,-0.030120,-0.999546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63969,6.00,1507,0,0,2024-03-05 09:00:00,0,0,0,0,0,0,1,0,0,-0.707107,7.071068e-01,0.623490,0.781831,0.436651,0.899631
63972,6.00,2280,0,0,2024-03-05 10:00:00,0,0,0,0,0,0,1,0,0,-0.866025,5.000000e-01,0.623490,0.781831,0.436651,0.899631
63975,7.00,3216,0,0,2024-03-05 11:00:00,0,0,0,0,0,0,1,0,0,-0.965926,2.588190e-01,0.623490,0.781831,0.436651,0.899631
63978,6.80,2858,0,0,2024-03-05 12:00:00,0,0,0,0,0,0,1,0,0,-1.000000,1.224647e-16,0.623490,0.781831,0.436651,0.899631


In [1]:
plt.plot(pd.to_datetime(df.date), df.pedestrians_count)

NameError: name 'plt' is not defined

In [17]:
matplotlib.use("TkAgg")
plt.figure()
plt.plot([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df['date']], df['pedestrians_count'])
plt.show()

plt.figure()
plt.hist(df['pedestrians_count'], bins=50)
plt.show()

window_size = 24*365  # You can adjust this as needed
m_avg = df['pedestrians_count'].rolling(window=window_size, center=True).mean()

# Plot original data and centered moving average
plt.figure()
plt.plot([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df['date']], df['pedestrians_count'], label='Pedestrians Count')
plt.plot([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df['date']], m_avg, label='Moving Average', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Pedestrians Count')
plt.title('Pedestrians Count with Centered Moving Average')
plt.legend()
plt.show()

In [4]:
#split train-test manually
df_unseen = df[[datetime.strptime(d, '%Y-%m-%d %H:%M:%S') <= datetime(2022, 5, 5) for d in df['date']]]
df_seen = df[[datetime.strptime(d, '%Y-%m-%d %H:%M:%S') > datetime(2022, 5, 5) for d in df['date']]]


X_train = df_seen.drop(columns=["date", 'pedestrians_count'])
y_train = df_seen['pedestrians_count']

X_test = df_unseen.drop(columns=["date", 'pedestrians_count'])
y_test = df_unseen['pedestrians_count']

In [31]:
forest = RandomForestRegressor(verbose=True, n_estimators=100, max_features=10)
forest.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    6.2s finished


RandomForestRegressor(max_features=10, verbose=True)

In [32]:
print("Score in the training set:" , forest.score(X_train, y_train))
print("Score in the test set:" , forest.score(X_test, y_test))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Score in the training set: 0.9874192740998712
Score in the test set: 0.8524516150566845


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [36]:
print(mean_squared_error(y_test, forest.predict(X_test), squared=False))
print(mean_absolute_error(y_test, forest.predict(X_test)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


542.0893225238132
287.5428252148997


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [37]:
print(mean_squared_error(y_train, forest.predict(X_train), squared=False))
print(mean_absolute_error(y_train, forest.predict(X_train)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


175.65253828529447
71.38109677017084


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


In [17]:
reals = np.array(df_unseen['pedestrians_count'])
preds = np.array(forest.predict(df_unseen.drop(columns=['pedestrians_count', 'date'])))
dates = np.array([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df_unseen['date']])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:    2.0s finished


In [12]:
%matplotlib tk
matplotlib.use("TkAgg")
plt.figure()
plt.plot(dates, reals)
plt.plot(dates, preds)
plt.show()

In [13]:
forest.oob_prediction_

array([ 399.36842105,  120.19444444,  654.46341463, ..., 1894.25      ,
        979.97435897, 1532.94444444])